# Raspagem de dados - Jornal Oficial de Mossoró

A ideia desse material é de capturar as informações de todos os arquivos do site do [Jornal Oficial de Mossoró](http://jom.prefeiturademossoro.com.br).

A versão do Python esperada é no mínimo a 3.8 devido a algumas funcionalidades utilizadas como a [expressão de atribuição/operador walrus](https://realpython.com/python-walrus-operator/)

As dependencias que precisaremos para fazer todo o trabalho são:
- httpx: responsavel por lidar com as requisições
- parse: nos permite fazer buscas a partir de padrões
- pyquery: nos permite fazer buscar no estilo jquery

In [ ]:
pip install -r requirements.txt

Importando o que vamos precisar usar

In [2]:
import httpx
from pyquery import PyQuery
from parse import search

from typing import List, Dict

In [3]:
def pegar_html_da_pagina_edicoes_por_numero_da_pagina(numero_da_pagina: int) -> str:
    jom_url_base = "http://jom.prefeiturademossoro.com.br/"
    jom_url_edicoes = f"{jom_url_base}todas-as-edicoes-2/"
    jom_url_edicoes_por_pagina = f"{jom_url_edicoes}?_page={numero_da_pagina}"

    jom_edicoes_response = httpx.get(jom_url_edicoes_por_pagina)
    
    return jom_edicoes_response.text

In [4]:
def pegar_lista_de_urls_de_jornais_por_pagina_de_edicao(html_pagina: str):
    classe_da_div_com_os_documentos = ".pt-cv-ifield"

    html_para_busca = PyQuery(html_pagina)
    gerador_de_elementos = html_para_busca(classe_da_div_com_os_documentos)
    
    lista_de_elementos = [item for item in gerador_de_elementos.items()]

    return [
        href for elem in lista_de_elementos
        if (href := elem("a:first").attr("href"))
    ]

In [5]:
def pegar_lista_de_todas_as_urls_de_paginas_de_jornais():
    todas_as_urls = []
    numero_da_pagina = 1

    while ((pagina_html := pegar_html_da_pagina_edicoes_por_numero_da_pagina(numero_da_pagina)) and
           (urls := pegar_lista_de_urls_de_jornais_por_pagina_de_edicao(pagina_html))):
        todas_as_urls += urls
        numero_da_pagina += 1

    return todas_as_urls

In [6]:
def pegar_html_da_pagina_de_documento_por_url(url_da_pagina_do_arquivo: str) -> str:
    jom_arquivo_response = httpx.get(url_da_pagina_do_arquivo)
    
    return jom_arquivo_response.text

In [7]:
def pegar_informacoes_do_arquivos_na_pagina(html_pagina: str) -> Dict:
    classe_da_div_com_as_informacoes = "div.entry-content:first"

    html_para_busca = PyQuery(html_pagina)
    gerador_de_elementos = html_para_busca(classe_da_div_com_as_informacoes)
    
    # Temos que iterar um generator para que possamos pegar o unico elemento
    # que queremos da pagina
    elemento = [item for item in gerador_de_elementos.items()][0]
    elemento_em_texto = elemento.text()
    
    return {
        "link_download": elemento("iframe:first").attr("src"),
        # Algumas paginas possuem a formatação de data errada:
        # <dia> de <mes> de <ano>: padrão
        # <dia> de <mes> <ano>: http://jom.prefeiturademossoro.com.br/2016/05/18/jom-n-356a/
        # <dia> <mes> de <ano>: http://jom.prefeiturademossoro.com.br/2014/02/14/jom-n-240/
        "data": (search("Data: {dia:d} de {mes:S} de {ano:d}", elemento_em_texto) or 
                 search("Data: {dia:d} de {mes:S} {ano:d}", elemento_em_texto) or
                 search("Data: {dia:d} {mes:S} de {ano:d}", elemento_em_texto)).named,
        "ano": search("Ano: {:S}", elemento_em_texto)[0],
        # Algumas páginas possuem nomes diferentes edição/número:
        # Número: padrão
        # Edição nº: http://jom.prefeiturademossoro.com.br/2008/05/09/jom-n-o-07a/
        "numero": (search("Número: {:w}", elemento_em_texto) or
                   search("Edição nº: {:w}", elemento_em_texto))[0],
        # Algumas paginas possuem o numero de paginas errado:
        # N.º de páginas: padrão
        # N.º Páginas: http://jom.prefeiturademossoro.com.br/2008/05/09/jom-n-o-07a/
        # Páginas: http://jom.prefeiturademossoro.com.br/2008/01/30/jom-n-o-01-ano-1-quarta-feira-30-de-janeiro-de-2008/
        # "XX": http://jom.prefeiturademossoro.com.br/2019/11/22/jom-n-o-537/
        "qnt_paginas": (search("N.º de páginas: {:d}", elemento_em_texto) or
                        search("N.º Páginas: {:d}", elemento_em_texto) or
                        search("Páginas: {:d}", elemento_em_texto) or
                        [None])[0],
    }

In [12]:
def pegar_lista_de_informacoes_dos_documentos_por_lista_de_urls(lista_de_urls: List) -> List:
    return [
        pegar_informacoes_do_arquivos_na_pagina(
            pegar_html_da_pagina_de_documento_por_url(url_documento)
        )
        for url_documento in lista_de_urls
    ]

In [10]:
lista_url_paginas_documentos = pegar_lista_de_todas_as_urls_de_paginas_de_jornais()

In [15]:
len(lista_url_paginas_documentos)

1394

In [14]:
lista_url_paginas_documentos

['http://jom.prefeiturademossoro.com.br/2021/10/16/jom-no-636a/',
 'http://jom.prefeiturademossoro.com.br/2021/10/15/jom-no-636/',
 'http://jom.prefeiturademossoro.com.br/2021/10/14/jom-no-635d-2/',
 'http://jom.prefeiturademossoro.com.br/2021/10/14/jom-no-635c-2/',
 'http://jom.prefeiturademossoro.com.br/2021/10/13/jom-no-635b/',
 'http://jom.prefeiturademossoro.com.br/2021/10/12/jom-no-635a/',
 'http://jom.prefeiturademossoro.com.br/2021/10/08/jom-no-635/',
 'http://jom.prefeiturademossoro.com.br/2021/10/07/jom-no-634d/',
 'http://jom.prefeiturademossoro.com.br/2021/10/06/jom-no-634c/',
 'http://jom.prefeiturademossoro.com.br/2021/10/05/jom-no-634b/',
 'http://jom.prefeiturademossoro.com.br/2021/10/04/jom-no-634a/',
 'http://jom.prefeiturademossoro.com.br/2021/10/01/jom-no-634/',
 'http://jom.prefeiturademossoro.com.br/2021/09/29/jom-no-633c/',
 'http://jom.prefeiturademossoro.com.br/2021/09/28/jom-no-633b/',
 'http://jom.prefeiturademossoro.com.br/2021/09/27/jom-no-633a/',
 'http://

In [13]:
lista_de_informacoes_documentos = pegar_lista_de_informacoes_dos_documentos_por_lista_de_urls(
    lista_url_paginas_documentos
)

In [16]:
len(lista_de_informacoes_documentos)

1394

In [17]:
lista_de_informacoes_documentos

[{'link_download': 'http://jom.prefeiturademossoro.com.br/wp-content/plugins/algori-pdf-viewer/dist/web/viewer.html?file=http%3A%2F%2Fjom.prefeiturademossoro.com.br%2Fwp-content%2Fuploads%2F2021%2F10%2F636a.pdf',
  'data': {'dia': 16, 'mes': 'outubro', 'ano': 2021},
  'ano': 'XIII',
  'numero': '636A',
  'qnt_paginas': 17},
 {'link_download': 'http://jom.prefeiturademossoro.com.br/wp-content/plugins/algori-pdf-viewer/dist/web/viewer.html?file=http%3A%2F%2Fjom.prefeiturademossoro.com.br%2Fwp-content%2Fuploads%2F2021%2F10%2F636.pdf',
  'data': {'dia': 15, 'mes': 'outubro', 'ano': 2021},
  'ano': 'XIII',
  'numero': '636',
  'qnt_paginas': 7},
 {'link_download': 'http://jom.prefeiturademossoro.com.br/wp-content/plugins/algori-pdf-viewer/dist/web/viewer.html?file=http%3A%2F%2Fjom.prefeiturademossoro.com.br%2Fwp-content%2Fuploads%2F2021%2F10%2F635d.pdf',
  'data': {'dia': 14, 'mes': 'outubro', 'ano': 2021},
  'ano': 'XIII',
  'numero': '635D',
  'qnt_paginas': 12},
 {'link_download': 'http: